# HW4: Bayesian Mixture Models


**STATS271/371: Applied Bayesian Statistics**

_Stanford University. Winter, 2021._

---

**Name:** _Your name here_

**Names of any collaborators:** _Names here_

*Due: 11:59pm Monday, May 3, 2021 via GradeScope*

---

In this homework assignment we will investigate image segmentation ---specifically, separating the background from the foreground of the image. To do so, you'll learn Bayesian mixtures of Gaussians using Gibbs sampling.

### Background: Image Segmentation
The figure below shows the original input image and the resulting segmentations into background and foreground. By the end of this assignment, you will have implemented the algorithm to achieve this segmentation.

Reference on image segmentation: https://en.wikipedia.org/wiki/Image_segmentation

![Fox](images/fox_seg.png "Segmentation of fox image")


### Background: Non-Bayesian Mixtures

To set the stage, we begin with a straightforward finite mixture model to cluster the pixels (with the number of clusters $K = 2$ in our image segmentation problem). The likelihood of the model is defined as a mixture of Gaussian distributions.

$$
p(x_n \mid z_n, \{\mu_k, \Sigma_k\}_{k=1}^K) = \mathcal{N}(x_n; \mu_{z_n}, \Sigma_{z_n})
$$

where $x_n \in \mathbb{R}^D$ is distributed according to a Gaussian distribution with the specified mean, $\mu_k$, and covariance, $\Sigma_k$, for its correspondign cluster $z_n = k$. We will represent the images as a set of $N$ pixels, $\{x_n\}_{n=1}^N$, each in $D=3$ dimensional space, since there are three color channels (red, green, and blue).

### Bayesian Mixture

We specify the following priors on $\mu_k$, $\Sigma_k$, and $\pi$.
- Assume a normal-inverse-Wishart prior prior for each cluster mean and covariance.
\begin{align} 
p(\mu_k, \Sigma_k) &= \mathrm{IW}(\Sigma_k \mid \Psi_0, \nu_0) \, \mathcal{N}(\mu_k \mid m_0, \kappa_0^{-1} \Sigma_k)
\end{align}
Here $\Psi_0, \nu_0, m_0, \kappa_0$ are hyper-parameters.

- We give a symmetric Dirichlet distribution prior to the mixing proportions, $\pi$:
$$ 
p(\pi \mid \alpha) = \text{Dirichlet}(\alpha)
$$
with the hyper-parameter $\alpha$.

## Problem 1 [math]: posterior calculations
In this problem, you will derive the conditional posterior distributions of the various model parameters.

These are related to the joint posterior distribution:
$$
p(\pi, \{\mu_k, \Sigma_k\}_{k=1}^K, \{z_n\}_{n=1}^N \mid \{x_n\}_{n=1}^N, \alpha, \nu_0, \Psi_0, m_0, \kappa_0) 
\propto p(\pi \mid \alpha) \prod_{k=1}^K p(\mu_k, \Sigma_k \mid \nu_0, \Psi_0, m_0, \kappa_0) \prod_{n=1}^N p(z_n \mid \pi) p(x_n \mid \mu_{z_n}, \Sigma_{z_n})
$$


You will need these derivations to be correct for the implementation in Problem 2 to be correct, so we highly recommend taking the time to double check them.

### (a) Derive the complete conditional for the parameters $\mu_k, \Sigma_k$.
It should be a normal-inverse-Wishart distribution, like the prior.

The conditional distributions for $z_n$ and $\pi$ are the same as shown in lecture.

## Problem 2: Gibbs Sampler
We have provided starter code below.
- First, you need to fill it with your own implementation of the Gibbs sampling algorithm. You may not rely on external implementations such as those offered by Tensorflow or scikit-learn.
- Then, you will test your code on a simple example.

### (a) Implement the Gibbs Sampler 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gibbs(X, K=2, 
          n_iter=100, 
          alpha=np.ones(2), 
          m0=np.zeros(3), 
          kappa0=1.0, 
          nu0=1.0, 
          Psi0=np.eye(3)):
    """
    Run the Gibbs samlper for a mixture of Gaussians with a NIW prior.
    
    Input:
    - X: Matrix of size (N, D). Each row of X stores one data point
    - K: the desired number of clusters in the model. Default: 2
    - n_iter: number of iterations of Gibbs sampling. Default: 100
    - alpha: hyperparameter of the Dirichlet prior on \pi.
    - m0, kappa0, nu0, Psi0, hyperparameters of normal-inverse-Wishart prior.
        
    Returns: 
    - log joint probability for each iteration
    - samples of parameters and assignments over iterations
    
    You will use these values later on for convergence diagnostics.
    """
    
    # TODO: Implement the Gibbs Sampler

### (b) test your implementation 
Test your example on a synthetic data set.

For example, the ground truth could be two clusters, with means [5,5,5] and [8,8,8] respectively. You could generate $50$ points in each cluster. 

Whichever example you choose, be sure to specify it and show that your implementation roughly recovers the ground truth by displaying the cluster means/covariances and trace plots of the log joint probability.

## Problem 3 : Perform image segmentation
Now that you have implemented the Gibbs Sampler, you are ready to perform image segmentation!
First, we define helper code to load the images and save the segmentation.

In [ ]:
def load_image(filename):
    # Read in the image, dropping the alpha channel
    image = plt.imread(filename + ".png")[:, :, :3]
    plt.imshow(image)

    # get height, width and number of channels
    H, W, C = image.shape
    X = image.copy().astype(float)

    # reshape into pixels, each has 3 channels (RGB)
    X = X.reshape((H * W, C)) 
    return image, X

def save_segmentation(image, assignments, filename=None):
    fig, axs = plt.subplots(1, K + 1, figsize=(4 * (K + 1), 4))
    axs[0].imshow(image)
    axs[0].set_axis_off()
    axs[0].set_title("original image")
    
    for k in range(K):
        im = image.copy()
        im[assignments != k] = np.nan
        axs[k+1].imshow(im)
        axs[k+1].set_axis_off()
        axs[k+1].set_title("component {}".format(k))
    
    if filename is not None:
        plt.savefig(filename)

#### Run the Gibbs sampler
Run the Gibbs sampler on the 4 provided images: "cow", "fox", "owl", "zebra". The hyper-parameters values should be the default ones from the Gibbs function.

Each sample of $\{z_n\}_{n=1}^N$ is a segmentation. To aggregate them into one sample, you could assign each pixel to the component it was most often attributed to in your samples.

## Problem 4: Diagnostics
### (a)
Make trace plots of the log joint probability and posterior marginals of the cluster means (e.g. as histograms for R, G, and B weights.)

### (b)
(Approximately) how many iterations are needed for convergence? Does this depend on the input image and/or initialization of the model parameters?


## Problem 5: Extensions

### (a) Sample the posterior predictive distribution to get a new image
Plot the resulting images.  

### (b) Improvements to the model
The generated images should not look realistic. Explain why that is the case: suggest improvements to the generative model that would make for more realistic samples.
You do not need to implement the change though.

### (c) Label switching
What could go wrong with the proposed method of deriving segmentations in problem 3? 

# Submission Instructions


**Formatting:** check that your code does not exceed 80 characters in line width. If you're working in Colab, you can set _Tools &rarr; Settings &rarr; Editor &rarr; Vertical ruler column_ to 80 to see when you've exceeded the limit. 

Download your notebook in .ipynb format and use the following commands to convert it to PDF:
```
jupyter nbconvert --to pdf hw4_yourname.ipynb
```

**Dependencies:**

- `nbconvert`: If you're using Anaconda for package management, 
```
conda install -c anaconda nbconvert
```

**Upload** your .ipynb and .pdf files to Gradescope. 